In [ ]:
!pip install edgar edgartools sec-edgar-downloader dotenv


In [ ]:
# Import necessary libraries
import pandas as pd
import re
from edgar import *
import tqdm
import os
from dotenv import load_dotenv

In [ ]:
# Load environment variables and set identity
load_dotenv()
user_name = os.getenv("USER_NAME")
user_email = os.getenv("USER_EMAIL")
set_identity(f"{user_name} {user_email}")


In [ ]:
# Set your identity for EDGAR access
set_identity(f"{user_name}" f"{user_email}")

In [ ]:
# # Understanding methods the library has to offer
# ticker='AAPL'
# form_type="10-K"
# year=2024
# filings=pd.DataFrame(columns=['ticker','date_filed','filing_url','accession_no','text'])
# company=Company(ticker)
# company
# company.get_filings(year=2024,form="10-K")
# yearly_filings=company.get_filings(form="10-K")
# apple_filing=yearly_filings.to_pandas()

In [ ]:
# Used Chat-GPT to generate the keywords inorder to filter and preserve 10-K filings involving ESG only to stay within the scope.
esg_keywords = {
            'environmental': [
                'climate', 'carbon', 'emissions?', 'greenhouse gas', 'ghg',
                'environmental', 'sustainability', 'renewable', 'pollution',
                'waste', 'energy efficiency', 'green', 'water', 'biodiversity'
            ],
            'social': [
                'diversity', 'equity', 'inclusion', 'dei', 'human rights',
                'labor', 'workforce', 'employee', 'community', 'social responsibility',
                'safety', 'health', 'wellbeing', 'training', 'culture'
            ],
            'governance': [
                'governance', 'esg', 'board composition', 'ethics', 'compliance',
                'risk management', 'stakeholder', 'transparency', 'audit',
                'accountability', 'executive compensation', 'oversight'
            ]
        }


Testing the code for one company "Apple"

In [ ]:
c = Company("AAPL")
filings = c.get_filings(form='10-K')

In [ ]:
filings

╭──────────────────────────────────────── Filings for Apple Inc. [320193] ────────────────────────────────────────╮
│                                                                                                                 │
│                                                                              Filing                             │
│    Form        Description                                                   Date         Accession Number      │
│  ─────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│    10-K        Annual report for public companies                            2024-11-01   0000320193-24-0001…   │
│    10-K        Annual report for public companies                            2023-11-03   0000320193-23-0001…   │
│    10-K        Annual report for public companies                            2022-10-28   0000320193-22-0001…   │
│    10-K        Annual report for public companies                     

In [ ]:
df = pd.DataFrame([
    {
        "ticker": "AAPL",
        "filing_date": pd.to_datetime(f.filing_date, errors="coerce"),
        "report_date": getattr(f, "report_date", None),
        "accession_no": getattr(f, "accession_no", None),
        "text": f.text() if hasattr(f, "text") else None
    }
    for f in filings
    if 2015 <= pd.to_datetime(getattr(f, "filing_date", None), errors="coerce").year <= 2025
])
df = df.dropna(subset=["filing_date"]).sort_values("filing_date").reset_index(drop=True)
pattern = re.compile("|".join(map(re.escape, esg_keywords)), re.IGNORECASE)
df["has_esg_keyword"] = df["text"].apply(lambda t: bool(pattern.search(t)) if isinstance(t, str) else False)

In [ ]:
df['text'][0]

'                                                                                                                   \n   UNITED STATES                                                                                                   \n                                                                                                                   \n                                                                                                                   \n   SECURITIES AND EXCHANGE COMMISSION                                                                              \n                                                                                                                   \nWashington, D.C. 20549\n\nForm 10-K\n\n(Mark One)\n\nxANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\nFor the fiscal year ended September 26, 2015\n\nor\n\n¨TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\nFor the 

In [ ]:
df.head()

,ticker,filing_date,report_date,accession_no,text,has_esg_keyword
0,AAPL,2015-10-28,2015-09-26,0001193125-15-356351,...,True
1,AAPL,2016-10-26,2016-09-24,0001628280-16-020309,...,True
2,AAPL,2017-11-03,2017-09-30,0000320193-17-000070,...,True
3,AAPL,2018-11-05,2018-09-29,0000320193-18-000145,...,True
4,AAPL,2019-10-31,2019-09-28,0000320193-19-000119,...,True


In [ ]:
df[df["has_esg_keyword"]==False]

,ticker,filing_date,report_date,accession_no,text,has_esg_keyword


In [ ]:
# print(df['text'][0])

Implementing the same block of code for the rest of the tickers

In [ ]:
#BRK-B ticker is not recognized in the EDGAR database, hence it is commented out
tickers = [
        'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', #'BRK-B',
        'UNH', 'XOM', 'JNJ', 'JPM', 'V', 'PG', 'MA', 'HD', 'CVX', 'MRK',
        'ABBV', 'KO', 'AVGO', 'PEP', 'COST', 'WMT', 'MCD', 'CSCO', 'TMO',
        'ACN', 'ABT', 'DHR', 'NKE', 'CRM', 'VZ', 'ADBE', 'CMCSA', 'WFC',
        'TXN', 'NEE', 'LIN', 'PM', 'NFLX', 'UPS', 'RTX', 'AMD', 'QCOM',
        'ORCL', 'HON', 'SPGI', 'LOW', 'INTU'
    ]


In [ ]:
def fetch_data(ticker: str, year: int) -> pd.DataFrame:
    company = Company(ticker)
    filings = company.get_filings(form="10-K")
    records = []
    for f in tqdm.tqdm(filings, desc=f"Fetching filings for {ticker}"):
        filing_date = pd.to_datetime(getattr(f, "filing_date", None), errors="coerce")
        if filing_date is pd.NaT:
            continue
        if not (2015 <= filing_date.year <= 2025):
            continue
        text = f.text() if hasattr(f, "text") else None
        has_esg = bool(re.compile("|".join(map(re.escape, esg_keywords)), re.IGNORECASE).search(text)) if isinstance(text, str) else False
        records.append({
            "ticker": ticker,
            "filing_date": filing_date,
            "report_date": getattr(f, "report_date", None),
            "accession_no": getattr(f, "accession_no", None),
            "text": text,
            "has_esg_keyword": has_esg
        })
    df = pd.DataFrame.from_records(records)
    df = df.sort_values("filing_date").reset_index(drop=True)
    return df

In [ ]:
all_filings=pd.concat([fetch_data(t,2015)for t in tickers],ignore_index=True)

Fetching filings for AAPL:   0%|          | 0/31 [00:00<?, ?it/s]

Fetching filings for AAPL:   3%|▎         | 1/31 [00:07<03:36,  7.21s/it]

Fetching filings for AAPL:   6%|▋         | 2/31 [00:12<03:02,  6.30s/it]

Fetching filings for AAPL:  10%|▉         | 3/31 [00:15<02:10,  4.66s/it]

Fetching filings for AAPL:  13%|█▎        | 4/31 [00:18<01:45,  3.90s/it]

...

Fetching filings for INTU:  25%|██▌       | 7/28 [01:16<04:58, 14.23s/it]

Fetching filings for INTU:  29%|██▊       | 8/28 [01:21<03:48, 11.45s/it]

Fetching filings for INTU:  32%|███▏      | 9/28 [01:28<03:09, 10.00s/it]

Fetching filings for INTU:  36%|███▌      | 10/28 [01:33<02:32,  8.47s/it]

Fetching filings for INTU: 100%|██████████| 28/28 [01:39<00:00,  3.56s/it]


In [ ]:
all_filings.to_csv("all_filings.csv",index=False)

In [ ]:
all_filings.head()

,ticker,filing_date,report_date,accession_no,text,has_esg_keyword
0,AAPL,2015-10-28,2015-09-26,0001193125-15-356351,...,True
1,AAPL,2016-10-26,2016-09-24,0001628280-16-020309,...,True
2,AAPL,2017-11-03,2017-09-30,0000320193-17-000070,...,True
3,AAPL,2018-11-05,2018-09-29,0000320193-18-000145,...,True
4,AAPL,2019-10-31,2019-09-28,0000320193-19-000119,...,True


In [ ]:
all_filings['ticker'].nunique()

49

In [ ]:
all_filings['text'].nunique()

543

In [ ]:
all_filings.shape

(543, 6)

In [ ]:
all_filings.to_csv("all_filings_esg_data_final.csv",index=False)